In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
product = pd.read_csv('skincare.csv')
product

,Unnamed: 0,label,url,brand,name,skin_type,price,concern,img
0,0,face-moisturizer,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,The Dewy Skin Cream Plumping & Hydrating Refil...,Normal and Dry,$72.00,"Dryness, Dullness,Fine Lines & Wrinkles,",https://www.sephora.com/productimages/sku/s278...
1,1,face-moisturizer,https://www.sephora.com/product/glow-recipe-pl...,Glow Recipe,Plum Plump Refillable Hyaluronic Acid Moisturizer,"Normal, Dry, Combination, and Oily",$40.00,"Dryness, Dullness,Loss of Firmness,Elasticity",https://www.sephora.com/productimages/sku/s253...
2,2,face-moisturizer,https://www.sephora.com/product/drunk-elephant...,Drunk Elephant,Lala Retro™ Nourishing Whipped Refillable Mois...,"Normal, Dry, Combination, and Oily",$64.00,"Dryness, Fine Lines and Wrinkles,Loss of Firmn...",https://www.sephora.com/productimages/sku/s223...
3,3,face-moisturizer,https://www.sephora.com/product/kale-spinach-h...,Youth To The People,Superfood Air-Whip Lightweight Face Moisturize...,"Normal, Combination, and Oily",$48.00,"Fine Lines and Wrinkles, Dryness,Loss of Firmn...",https://www.sephora.com/productimages/sku/s186...
4,4,face-moisturizer,https://www.sephora.com/product/clarins-multi-...,Clarins,"Multi-Active Day Moisturizer for Lines, Pores,...","Normal, Dry, Combination, and Oily",$59.00,"Fine Lines and Wrinkles, Pores,Dullness,",https://www.sephora.com/productimages/sku/s273...
...,...,...,...,...,...,...,...,...,...
731,731,eye-cream,https://www.sephora.com/product/plantscription...,Origins,Plantscription™ Anti-Aging Power Eye Cream,"Normal, Dry, Combination, and Oily",$66.00,"Fine Lines and Wrinkles,,,",https://www.sephora.com/productimages/sku/s167...
732,732,eye-cream,https://www.sephora.com/product/cucumber-de-to...,Peter Thomas Roth,Cucumber De-Tox™ Hydra-Gel Eye Patches,"Normal, Dry, Combination, and Oily",$55.00,"Fine Lines and Wrinkles, Dryness,Puffiness,",https://www.sephora.com/productimages/sku/s186...
733,733,eye-cream,https://www.sephora.com/product/glopro-eye-mic...,BeautyBio,GloPRO® EYE MicroTip™ Attachment Head,"Normal, Dry, Combination, and Oily",$39.00,"Fine Lines and Wrinkles, Dullness and Uneven T...",https://www.sephora.com/productimages/sku/s216...
734,734,eye-cream,https://www.sephora.com/product/guerlain-abeil...,GUERLAIN,Abeille Royale Anti-Aging Eye Cream,"Normal, Dry, Combination, and Oily",$110.00,"Fine Lines and Wrinkles, Dullness and Uneven T...",https://www.sephora.com/productimages/sku/s235...


In [3]:
product['brand'] = product['brand'].str.lower()
product['name'] = product['name'].str.lower()
product['skin_type'] = product['skin_type'].str.lower()
product['skin_type'] = product['skin_type'].str.replace(' and ', ',').str.replace(' or ', ',')
product['concern'] = product['concern'].str.lower()
product['concern'] = product['concern'].str.replace(' and ', ',').str.replace(' or ', ',')
product

,Unnamed: 0,label,url,brand,name,skin_type,price,concern,img
0,0,face-moisturizer,https://www.sephora.com/product/the-dewy-skin-...,tatcha,the dewy skin cream plumping & hydrating refil...,"normal,dry",$72.00,"dryness, dullness,fine lines & wrinkles,",https://www.sephora.com/productimages/sku/s278...
1,1,face-moisturizer,https://www.sephora.com/product/glow-recipe-pl...,glow recipe,plum plump refillable hyaluronic acid moisturizer,"normal, dry, combination,,oily",$40.00,"dryness, dullness,loss of firmness,elasticity",https://www.sephora.com/productimages/sku/s253...
2,2,face-moisturizer,https://www.sephora.com/product/drunk-elephant...,drunk elephant,lala retro™ nourishing whipped refillable mois...,"normal, dry, combination,,oily",$64.00,"dryness, fine lines,wrinkles,loss of firmness,...",https://www.sephora.com/productimages/sku/s223...
3,3,face-moisturizer,https://www.sephora.com/product/kale-spinach-h...,youth to the people,superfood air-whip lightweight face moisturize...,"normal, combination,,oily",$48.00,"fine lines,wrinkles, dryness,loss of firmness,...",https://www.sephora.com/productimages/sku/s186...
4,4,face-moisturizer,https://www.sephora.com/product/clarins-multi-...,clarins,"multi-active day moisturizer for lines, pores,...","normal, dry, combination,,oily",$59.00,"fine lines,wrinkles, pores,dullness,",https://www.sephora.com/productimages/sku/s273...
...,...,...,...,...,...,...,...,...,...
731,731,eye-cream,https://www.sephora.com/product/plantscription...,origins,plantscription™ anti-aging power eye cream,"normal, dry, combination,,oily",$66.00,"fine lines,wrinkles,,,",https://www.sephora.com/productimages/sku/s167...
732,732,eye-cream,https://www.sephora.com/product/cucumber-de-to...,peter thomas roth,cucumber de-tox™ hydra-gel eye patches,"normal, dry, combination,,oily",$55.00,"fine lines,wrinkles, dryness,puffiness,",https://www.sephora.com/productimages/sku/s186...
733,733,eye-cream,https://www.sephora.com/product/glopro-eye-mic...,beautybio,glopro® eye microtip™ attachment head,"normal, dry, combination,,oily",$39.00,"fine lines,wrinkles, dullness,uneven texture,l...",https://www.sephora.com/productimages/sku/s216...
734,734,eye-cream,https://www.sephora.com/product/guerlain-abeil...,guerlain,abeille royale anti-aging eye cream,"normal, dry, combination,,oily",$110.00,"fine lines,wrinkles, dullness,uneven texture,l...",https://www.sephora.com/productimages/sku/s235...


In [4]:
LABELS = list(product.label.unique())
LABELS

['face-moisturizer', 'cleanser', 'sunscreen', 'masks', 'eye-cream']

In [5]:
entries = len(product)
entries

736

In [8]:
def concern_elements(comma_sep_concerns):
    words = comma_sep_concerns.split(',')
    for w in words:
        if w != '':
            temp = w.strip()
            if temp in concerns:
                concerns[temp] += 1
            else:
                concerns[temp] = 1

# features
concerns = {}
for i in range(entries):
    concern_elements(product.iloc[i]['concern'])

print(sorted(concerns.items(), key=lambda kv:(kv[1], kv[0])))   

[('anti-aging', 1), ('dryness.', 1), ('loss if firmness', 1), ('pregnancy', 1), ('puffiness/dark circles', 1), ('dullness/uneven texture', 2), ('fine lines & wrinkles', 2), ('fines lines', 2), ('loss of firmness/elasticity', 2), ('acne/blemishes', 3), ('dark spot', 4), ('fine lines/wrinkles', 6), ('uneven skin tone', 7), ('acne', 49), ('puffiness', 50), ('blemishes', 56), ('dark circles', 61), ('dark spots', 67), ('redness', 90), ('oiliness', 108), ('pores', 161), ('elasticity', 179), ('loss of firmness', 180), ('uneven texture', 186), ('dullness', 292), ('fine lines', 292), ('wrinkles', 293), ('dryness', 437)]


In [9]:
concerns.pop('anti-aging')
concerns.pop('dryness.')
concerns.pop('loss if firmness')
concerns.pop('pregnancy')
concerns.pop('puffiness/dark circles')
concerns.pop('dullness/uneven texture')
concerns.pop('fine lines & wrinkles')
concerns.pop('fines lines')
concerns.pop('loss of firmness/elasticity')
concerns.pop('acne/blemishes')
concerns.pop('dark spot')
concerns.pop('fine lines/wrinkles')
concerns.pop('uneven skin tone')

7

In [10]:
def split_skin_types(df, skin_type_column='skin_type'):
    data = []

    # Function to split skin types into individual rows
    def expand_row(row):
        skin_types = row[skin_type_column]
        skin_types = skin_types.replace(',,', " ")
        skin_types = skin_types.replace(',', '')
        skin_types = skin_types.split()

        if 'combination' in skin_types:
            stype = 'combination'
        elif 'oily' in skin_types:
            stype = 'oily'
        elif 'dry' in skin_types:
            stype = 'dry'
        else:
            stype = 'normal'


        if stype is not None:
            row_copy = row.copy()
            row_copy[skin_type_column] = stype
            data.append(row_copy)

    # Apply the function to each row and collect expanded rows
    df.apply(expand_row, axis=1)

    # Create a DataFrame from the collected data
    expanded_df = pd.DataFrame(data)

    return expanded_df


product_expanded = split_skin_types(product)
product_expanded

,Unnamed: 0,label,url,brand,name,skin_type,price,concern,img
0,0,face-moisturizer,https://www.sephora.com/product/the-dewy-skin-...,tatcha,the dewy skin cream plumping & hydrating refil...,normal,$72.00,"dryness, dullness,fine lines & wrinkles,",https://www.sephora.com/productimages/sku/s278...
1,1,face-moisturizer,https://www.sephora.com/product/glow-recipe-pl...,glow recipe,plum plump refillable hyaluronic acid moisturizer,combination,$40.00,"dryness, dullness,loss of firmness,elasticity",https://www.sephora.com/productimages/sku/s253...
2,2,face-moisturizer,https://www.sephora.com/product/drunk-elephant...,drunk elephant,lala retro™ nourishing whipped refillable mois...,combination,$64.00,"dryness, fine lines,wrinkles,loss of firmness,...",https://www.sephora.com/productimages/sku/s223...
3,3,face-moisturizer,https://www.sephora.com/product/kale-spinach-h...,youth to the people,superfood air-whip lightweight face moisturize...,combination,$48.00,"fine lines,wrinkles, dryness,loss of firmness,...",https://www.sephora.com/productimages/sku/s186...
4,4,face-moisturizer,https://www.sephora.com/product/clarins-multi-...,clarins,"multi-active day moisturizer for lines, pores,...",combination,$59.00,"fine lines,wrinkles, pores,dullness,",https://www.sephora.com/productimages/sku/s273...
...,...,...,...,...,...,...,...,...,...
731,731,eye-cream,https://www.sephora.com/product/plantscription...,origins,plantscription™ anti-aging power eye cream,combination,$66.00,"fine lines,wrinkles,,,",https://www.sephora.com/productimages/sku/s167...
732,732,eye-cream,https://www.sephora.com/product/cucumber-de-to...,peter thomas roth,cucumber de-tox™ hydra-gel eye patches,combination,$55.00,"fine lines,wrinkles, dryness,puffiness,",https://www.sephora.com/productimages/sku/s186...
733,733,eye-cream,https://www.sephora.com/product/glopro-eye-mic...,beautybio,glopro® eye microtip™ attachment head,combination,$39.00,"fine lines,wrinkles, dullness,uneven texture,l...",https://www.sephora.com/productimages/sku/s216...
734,734,eye-cream,https://www.sephora.com/product/guerlain-abeil...,guerlain,abeille royale anti-aging eye cream,combination,$110.00,"fine lines,wrinkles, dullness,uneven texture,l...",https://www.sephora.com/productimages/sku/s235...


In [11]:
product_expanded['skin_type'].value_counts()

skin_type
combination    658
normal          56
dry             15
oily             7
Name: count, dtype: int64

In [15]:
features= list(product_expanded['skin_type'].unique()) + list(concerns)
print(len(features), features)

19 ['normal', 'combination', 'dry', 'oily', 'dryness', 'dullness', 'loss of firmness', 'elasticity', 'fine lines', 'wrinkles', 'pores', 'redness', 'uneven texture', 'dark spots', 'oiliness', 'acne', 'blemishes', 'dark circles', 'puffiness']


In [16]:
def search_concern(target, i):
    if target in product_expanded.iloc[i]['concern']:
        return True
    return False

one_hot_encodings = np.zeros([entries, len(features)])

#skin types first
for i in range(entries):
    for j in range(4):
        target = features[j]
        sk_type = product_expanded.iloc[i]['skin_type']
        if sk_type == 'combination':
            one_hot_encodings[i][0:4] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(len(features)):
        feature = features[j]
        if feature in product_expanded.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1

one_hot_encodings

array([[1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 1., 0.]])

In [17]:
x = one_hot_encodings[2]
print(list(x))

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [20]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)

In [22]:
# utility functions
def name2index(name):
    return product_expanded[product_expanded["name"]==name].index.tolist()[0]

def index2prod(index):
    return product_expanded.iloc[index]

def wrap(info_arr):
    result = {'brand': info_arr[0], 'name': info_arr[1], 'price': info_arr[2], 'url': info_arr[3],
              'skin_type': info_arr[4], 'concern': str(info_arr[5]).split(','), 'img': str(info_arr[6])}
    #     print(info_arr)
    return result

In [23]:
# recommend top 10 similar items from a category
def recs_cs(vector = None, name = None, label = None, count = 5):
    products = []
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    product_expanded['cs'] = cs_values[0]

    if label:
        dff = product_expanded[product_expanded['label'] == label]
    else:
        dff = product_expanded

    if name:
        dff = dff[dff['name'] != name]
    recommendations = dff.sort_values('cs', ascending=False).head(count)
    #   print(f"Top {count} matching {label} items")
    data = recommendations[['brand', 'name', 'price', 'url','skin_type', 'concern', 'img']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

In [24]:
# overall recommendation
def recs_essentials(vector = None, name = None):
    #     print("ESSENTIALS:")
    response = {}
    for label in LABELS:
        #         print(f"{label}:")
        if name:
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response

In [25]:
x = [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]

y = recs_essentials(x, None)
print(y)

{'face-moisturizer': [{'brand': 'fenty skin', 'name': 'hydra vizor refillable invisible face moisturizer spf 30 with niacinamide', 'price': '$40.00 ', 'url': 'https://www.sephora.com/product/fenty-skin-hydra-vizor-moisturizer-broad-spectrum-spf-30-sunscreen-P467249?icid2=products%20grid:p467249:product', 'skin_type': 'combination', 'concern': ['dark spots', ' pores', 'dryness', ''], 'img': 'https://www.sephora.com/productimages/sku/s2418879-main-zoom.jpg?pb=clean-at-sephora&imwidth=175'}, {'brand': 'fenty skin', 'name': 'hydra vizor mineral spf 30 refillable moisturizer', 'price': '$36.00 ', 'url': 'https://www.sephora.com/product/hydra-vizor-mineral-spf-30-refillable-moisturizer-P506080?icid2=products%20grid:p506080:product', 'skin_type': 'combination', 'concern': ['dark spots', ' pores', 'dryness', ''], 'img': 'https://www.sephora.com/productimages/sku/s2648764-main-zoom.jpg?pb=clean-at-sephora&imwidth=175'}, {'brand': 'murad', 'name': 'oil and pore control mattifying face sunscreen 

In [ ]:
product_expanded.to_csv('final_extended_dataset2.csv')

# Makeup Recommendation

In [26]:
makeup = pd.read_csv('makeup_dataset.csv')
makeup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   label        246 non-null    object
 1   url          246 non-null    object
 2   brand        246 non-null    object
 3   name         246 non-null    object
 4   skin_tone    246 non-null    object
 5   skin_type    54 non-null     object
 6   concern_1    90 non-null     object
 7   concern_2    36 non-null     object
 8   formulation  246 non-null    object
 9   price        246 non-null    object
 10  image        246 non-null    object
dtypes: object(11)
memory usage: 21.3+ KB


In [27]:
makeup.drop(columns = ['concern_1', 'concern_2', 'formulation'], inplace = True)
makeup

,label,url,brand,name,skin_tone,skin_type,price,image
0,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333698&icid2=products grid:p455418:product,ILIA,Super Serum Skin Tint SPF 40 Skincare Foundation,Very Light,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333698-main-zoom.jpg?imwidth=612
1,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333664,ILIA,Super Serum Skin Tint SPF 40 Skincare Foundation,Light,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333664-main-zoom.jpg?imwidth=612
2,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333607,ILIA,Super Serum Skin Tint SPF 40 Skincare Foundation,Medium,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333607-main-zoom.jpg?imwidth=612
3,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2428795,ILIA,Super Serum Skin Tint SPF 40 Skincare Foundation,Medium Deep,NaN,$48.00,https://www.sephora.com/productimages/sku/s2428795-main-zoom.jpg?imwidth=612
4,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333565,ILIA,Super Serum Skin Tint SPF 40 Skincare Foundation,Deep,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333565-main-zoom.jpg?imwidth=612
...,...,...,...,...,...,...,...,...
241,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2636900,Givenchy,Prisme Libre Skin-Caring 24H Hydrating + Radiant + Correcting Creamy Concealer,Light,NaN,$39.00,https://www.sephora.com/productimages/sku/s2636900-main-zoom.jpg?imwidth=315
242,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2636942,Givenchy,Prisme Libre Skin-Caring 24H Hydrating + Radiant + Correcting Creamy Concealer,Medium,NaN,$39.00,https://www.sephora.com/productimages/sku/s2636942-main-zoom.jpg?imwidth=315
243,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2637023,Givenchy,Prisme Libre Skin-Caring 24H Hydrating + Radiant + Correcting Creamy Concealer,Medium Deep,NaN,$39.00,https://www.sephora.com/productimages/sku/s2637023-main-zoom.jpg?imwidth=315
244,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2637056,Givenchy,Prisme Libre Skin-Caring 24H Hydrating + Radiant + Correcting Creamy Concealer,Deep,NaN,$39.00,https://www.sephora.com/productimages/sku/s2637056-main-zoom.jpg?imwidth=315


In [28]:
makeup['skin_tone'].value_counts()

skin_tone
Very Light     41
Light          41
Medium         41
Medium Deep    41
Deep           41
Extra Deep     41
Name: count, dtype: int64

In [29]:
makeup.isna().sum()

label          0
url            0
brand          0
name           0
skin_tone      0
skin_type    192
price          0
image          0
dtype: int64

In [30]:
makeup[makeup['label'] == 'foundation']['skin_type'].value_counts()

skin_type
Normal, Dry, Combination, and Oily     6
Normal, Combination, and Oily          6
Normal and Combination                 6
Combination and Oily                   6
Name: count, dtype: int64

In [31]:
makeup[makeup['label'] == 'foundation']['skin_type'].isna().sum()

102

In [32]:
makeup[makeup['label'] == 'primer']['skin_type'].isna().sum()

0

In [33]:
makeup[makeup['label'] == 'concealer']['skin_type'].isna().sum()

90

In [35]:
makeup[makeup['label'] == 'concealer']['skin_type'].value_counts()

skin_type
Normal, Dry, Combination, and Oily    24
Normal, Dry and Combination            6
Name: count, dtype: int64

In [36]:
makeup['brand'] = makeup['brand'].str.lower()
makeup['name'] = makeup['name'].str.lower()
makeup['skin_type'] = makeup['skin_type'].str.lower()
makeup['skin_tone'] = makeup['skin_tone'].str.lower()
makeup

,label,url,brand,name,skin_tone,skin_type,price,image
0,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333698&icid2=products grid:p455418:product,ilia,super serum skin tint spf 40 skincare foundation,very light,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333698-main-zoom.jpg?imwidth=612
1,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333664,ilia,super serum skin tint spf 40 skincare foundation,light,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333664-main-zoom.jpg?imwidth=612
2,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333607,ilia,super serum skin tint spf 40 skincare foundation,medium,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333607-main-zoom.jpg?imwidth=612
3,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2428795,ilia,super serum skin tint spf 40 skincare foundation,medium deep,NaN,$48.00,https://www.sephora.com/productimages/sku/s2428795-main-zoom.jpg?imwidth=612
4,foundation,https://www.sephora.com/product/ilia-super-serum-skin-tint-spf-40-P455418?skuId=2333565,ilia,super serum skin tint spf 40 skincare foundation,deep,NaN,$48.00,https://www.sephora.com/productimages/sku/s2333565-main-zoom.jpg?imwidth=612
...,...,...,...,...,...,...,...,...
241,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2636900,givenchy,prisme libre skin-caring 24h hydrating + radiant + correcting creamy concealer,light,NaN,$39.00,https://www.sephora.com/productimages/sku/s2636900-main-zoom.jpg?imwidth=315
242,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2636942,givenchy,prisme libre skin-caring 24h hydrating + radiant + correcting creamy concealer,medium,NaN,$39.00,https://www.sephora.com/productimages/sku/s2636942-main-zoom.jpg?imwidth=315
243,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2637023,givenchy,prisme libre skin-caring 24h hydrating + radiant + correcting creamy concealer,medium deep,NaN,$39.00,https://www.sephora.com/productimages/sku/s2637023-main-zoom.jpg?imwidth=315
244,concealer,https://www.sephora.com/product/givenchy-prisme-libre-skin-caring-24h-hydrating-radiant-correcting-creamy-concealer-P505129?skuId=2637056,givenchy,prisme libre skin-caring 24h hydrating + radiant + correcting creamy concealer,deep,NaN,$39.00,https://www.sephora.com/productimages/sku/s2637056-main-zoom.jpg?imwidth=315


In [37]:
def wrap_makeup(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['skin type'] = info_arr[4]
    result['skin tone'] = info_arr[5]
    return result


def makeup_recommendation(skin_tone):
    result = []
    dff = pd.DataFrame()
    dff = dff._append(makeup[(makeup['skin_tone'] == skin_tone) & (makeup['label'] == 'foundation')].head(3))
    dff = dff._append(makeup[(makeup['skin_tone'] == skin_tone) & (makeup['label'] == 'concealer')].head(3))
    # dff = dff._append(makeup[(makeup['skin tone'] == skin_tone) & (makeup['skin type'] == skin_type) & (makeup['label'] == 'primer')].head(2))
    dff= dff.sample(frac = 1)
    data = dff[['brand', 'name', 'price', 'url', 'skin_type', 'skin_tone']].to_dict('split')['data']
    for element in data:
        result.append(wrap_makeup(element))
    return result

In [38]:
makeup_recommendation('deep')

[{'brand': 'tower 28 beauty',
  'name': 'swipe all-over hydrating serum concealer',
  'price': '$22.00',
  'url': 'https://www.sephora.com/product/swipe-all-over-hydrating-serum-concealer-P507142?skuId=2697621&icid2=products%20grid:p507142:product',
  'skin type': nan,
  'skin tone': 'deep'},
 {'brand': 'haus labs by lady gaga',
  'name': 'triclone skin tech medium coverage foundation with fermented arnica',
  'price': '$45.00',
  'url': 'https://www.sephora.com/product/triclone-skin-tech-medium-coverage-foundation-with-fermented-arnica-P502185?skuId=2597102',
  'skin type': nan,
  'skin tone': 'deep'},
 {'brand': 'hourglass',
  'name': 'vanish™ airbrush concealer',
  'price': '$36.00',
  'url': 'https://www.sephora.com/product/hourglass-vanish-airbrush-concealer-P454042?skuId=2303071',
  'skin type': 'normal, dry, combination, and oily',
  'skin tone': 'deep'},
 {'brand': 'sephora',
  'name': 'reveal the real 12hr soft radiant skin tint',
  'price': '$22.00',
  'url': 'https://www.sep